In [395]:
import json
import requests
import pandas as pd
import numpy as np

In [396]:
f = open("dests.txt")

In [397]:
destinations_lst = [line.rstrip('\n') for line in f]
api_key = 'privat key :)'
origins ="Tel Aviv-Jaffa"

In [398]:
destinations_lst

['Istanbul', 'Amsterdam', 'Valletta', 'Basel', 'Doha']

In [399]:
Target=np.array([])
Distance=np.array([])
Duration=np.array([])
Longitude=np.array([])
latitude=np.array([])

### latitude & longitude

In [400]:
def get_lat_lng(address_string,api_key=api_key):
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (address_string,api_key)
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            return("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
                if not response_data["status"] == "OK":
                    return ("google API problem")
            except:
                return ("Response not in valid JSON format")
    except:
        return("Something went wrong with requests.get")
    lat=response_data['results'][0]["geometry"]["location"]['lat']
    lng=response_data['results'][0]["geometry"]["location"]['lng']

    
    return lat,lng

In [401]:
def distance_duration(destinations,origins=origins,api_key=api_key):
    url = "https://maps.googleapis.com/maps/api/distancematrix/json?origins=%s&destinations=%s&key=%s"% (origins,destinations,api_key)
    try:
        response = requests.get(url)
        if not response.status_code == 200:
            return("HTTP error",response.status_code)
        else:
            try:
                response_data = response.json()
                if not response_data["status"] == "OK":
                    return ("google API problem")
            except:
                return ("Response not in valid JSON format")
    except:
        return("Something went wrong with requests.get")
    dis=response_data['rows'][0]['elements'][0]['distance']['text']
    dur=response_data['rows'][0]['elements'][0]['duration']['text']
    
    return dis,dur

In [402]:
for city in destinations_lst:
    Target=np.append(Target,city)
    lat_lng=get_lat_lng(city)
    dis_dur=distance_duration(city)
    latitude=np.append(latitude,lat_lng[0])
    Longitude=np.append(Longitude,lat_lng[1])
    Distance=np.append(Distance,dis_dur[0])
    Duration=np.append(Duration,dis_dur[1])




In [403]:
d = {'Target': Target , 'Distance_km':Distance ,'Duration (hour+minutes)':Duration , 'Longitude': Longitude,'Latitude':latitude}
df = pd.DataFrame(data=d)

In [404]:
df

,Target,Distance_km,Duration (hour+minutes),Longitude,Latitude
0,Istanbul,"1,815 km",21 hours 3 mins,28.978359,41.008238
1,Amsterdam,"4,533 km",2 days 0 hours,4.904139,52.367573
2,Valletta,"3,793 km",2 days 3 hours,14.514100,35.899237
3,Basel,"4,093 km",1 day 20 hours,7.588576,47.559599
4,Doha,"2,164 km",22 hours 39 mins,51.531040,25.285447
